=> **LDA** (Latent Dirichlet Allocation): é um modelo estatístico usado para descobrir tópicos ocultos em grandes coleções de textos. O LDA assume que cada documento em um conjunto de textos é composto por uma mistura de tópicos e que cada tópico é representado por um conjunto de palavras. Ele funciona assim:

 - Escolhe aleatoriamente um conjunto de tópicos para cada documento.

 - Observa as palavras no documento e ajusta a distribuição de tópicos com base em quais palavras aparecem juntas frequentemente.

 - Após várias iterações, o modelo aprende a associar palavras que pertencem ao mesmo tópico.


In [ ]:
import pandas as pd

papers = pd.read_csv("papers.csv")

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from tqdm.auto import tqdm


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

tqdm.pandas()

def text_cleaner(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = word_tokenize(text)
    custom_stopwords = set(stopwords.words('english'))
    cleaned_tokens = [token for token in tokens if token not in custom_stopwords]
    cleaned_tokens = [token for token in cleaned_tokens if len(token) > 2]
    return ' '.join(cleaned_tokens)

papers['text_processed'] = papers['paper_text'].progress_apply(text_cleaner)

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

tfidf = CountVectorizer(
    lowercase=True,
    stop_words=stopwords.words('english'),
    min_df=10,
    max_df=0.95,
    max_features=5000
)

papers['tfidf'] = papers['text_processed'].progress_apply(lambda x: tfidf.transform([x]).toarray()[0])


tfidf.fit(papers['text_processed'])

AttributeError: 'Series' object has no attribute 'progress_apply'

In [ ]:
papers['tfidf'] = papers['text_processed'].progress_apply(lambda x: tfidf.transform([x]).toarray()[0])

In [ ]:
# Document-Term Matrix
dtm = pd.DataFrame(papers['tfidf'].to_list(),
                   columns=tfidf.get_feature_names_out())

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
import numpy as np

lda = LDA(n_components=5, random_state=42).fit(dtm.values)
topics = lda.transform(dtm.values)

In [ ]:
def get_model_topics(model, feature_names, n_top_words=5):
    word_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[idx] for idx in top_features_ind]
        word_dict[topic_idx] = top_features
    return pd.DataFrame(word_dict)

get_model_topics(lda, dtm.columns, 10)


In [ ]:
papers['topic'] =  np.argmax(topics, axis=1)

In [ ]:
papers.topic.value_counts()

In [ ]:
from wordcloud import WordCloud

import matplotlib.pyplot as plt

def draw_all_wordclouds(papers, num_topics=5):
    fig, axes = plt.subplots(1, num_topics, figsize=(20, 5))  # 1 linha, 5 colunas
    
    for i, ax in enumerate(axes):
        texts = ' '.join(papers[papers['topic'] == i]['text_processed'])
        wordcloud = WordCloud(width=800, height=800, background_color='white', min_font_size=10).generate(texts)

        ax.imshow(wordcloud, interpolation='bilinear')
        ax.axis('off')  # Remove os eixos
        ax.set_title(f"Tópico {i}")

    plt.tight_layout()
    plt.show()

# Chamada da função
draw_all_wordclouds(papers)